In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
path_to_csv = "kdd.txt"

df = pd.read_csv(path_to_csv, header = None)

In [4]:
fig = plt.figure()
# Quel graphique peut-on faire ? (cf. 1er point du sujet)

<Figure size 432x288 with 0 Axes>

In [5]:
df.rename(columns={0: 'duration', 
                   1: 'protocol_type',
                   2: 'service', 
                   3: 'flag',
                   4: 'src_bytes',
                   5: 'dst_bytes',
                   6: 'land',
                   7: 'wrong_fragment',
                   8: 'urgent',
                   9: 'hot',
                   10: 'num_failed_logins',
                   11: 'logged_in',
                   12: 'num_compromised',
                   13: 'root_shell',
                   14: 'su_attempted',
                   15: 'num_root',
                   16: 'num_file_creations',
                   17: 'num_shells',
                   18: 'num_access_files',
                   19: 'num_outbound_cmds',
                   20: 'is_host_login',
                   21: 'is_guest_login',
                   22: 'count',
                   23: 'srv_count',
                   24: 'serror_rate',
                   25: 'srv_serror_rate',
                   26: 'rerror_rate',
                   27: 'srv_rerror_rate',
                   28: 'same_srv_rate',
                   29: 'diff_srv_rate',
                   30: 'srv_diff_host_rate',
                   31: 'dst_host_count',
                   32: 'dst_host_srv_count',
                   33: 'dst_host_same_srv_rate',
                   34: 'dst_host_diff_srv_rate',
                   35: 'dst_host_same_src_port_rate',
                   36: 'dst_host_srv_diff_host_rate',
                   37: 'dst_host_serror_rate',
                   38: 'dst_host_srv_serror_rate',
                   39: 'dst_host_rerror_rate',
                   40: 'dst_host_srv_rerror_rate',
                   41: 'normal'
                  }, inplace=True)

In [4]:
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,normal
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.00,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.00,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.00,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.00,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.00,0.0,normal.
5,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.00,0.00,0.00,0.00,0.0,normal.
6,0,tcp,http,SF,212,1940,0,0,0,0,...,69,1.0,0.0,1.00,0.04,0.00,0.00,0.00,0.0,normal.
7,0,tcp,http,SF,159,4087,0,0,0,0,...,79,1.0,0.0,0.09,0.04,0.00,0.00,0.00,0.0,normal.
8,0,tcp,http,SF,210,151,0,0,0,0,...,89,1.0,0.0,0.12,0.04,0.00,0.00,0.00,0.0,normal.
9,0,tcp,http,SF,212,786,0,0,0,1,...,99,1.0,0.0,0.12,0.05,0.00,0.00,0.00,0.0,normal.


In [7]:
df.groupby(['hot']).size().reset_index(name='counts')

,hot,counts
0,0,490829
1,1,256
2,2,2192
3,3,38
4,4,112
5,5,51
6,6,104
7,7,5
8,9,1
9,10,1
